In [1]:
%load_ext autoreload
%autoreload 2
import time
from klampt import *
from klampt.math import vectorops,so3,se3
from klampt.io import loader
import math
import random
from IPython.display import clear_output,Markdown
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
fn = "data/ur5Blocks.xml"
res = world.readFile(fn)
robot = world.robot(0)

# perturb the block configuration
objectTransforms = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]
random.shuffle(objectTransforms)
for i in range(world.numRigidObjects()):
    R,t = objectTransforms[i]
    t[0] += random.uniform(-0.05,0.05)
    t[1] += random.uniform(-0.05,0.05)
    R = so3.from_axis_angle(((0,0,1),random.uniform(0,math.pi*2)))
    world.rigidObject(i).setTransform(R,t)

#set the home configuration
qhome  = robot.getConfig()
qhome[1] = 2.3562
qhome[2] = -math.pi/3.0
qhome[3] = math.pi*2/3.0
qhome[4] = 0.64181
qhome[5] = math.pi/2.0

kvis = ipyklampt.KlamptWidget(world,width=600,height=400)
kvis.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})


In [2]:
#set up the controller and useful subroutines for converting to/from Klampt
def controller_to_klampt(qcontroller):
    """Converts a 6+1 controller configuration to Klampt format"""
    qorig = robot.getConfig()
    q = [v for v in qorig]
    for i in range(6):
        q[i+1] = qcontroller[i]
    robot.setConfig(q)
    robot.driver(6).setValue(qcontroller[6])
    q = robot.getConfig()
    robot.setConfig(qorig)
    return q

def klampt_to_controller(qklampt):
    """Converts a Klamp't configuration to 6+1 controller format"""
    qorig = robot.getConfig()
    res = [qklampt[i+1] for i in range(6)]
    robot.setConfig(qklampt)
    res.append(robot.driver(6).getValue())
    robot.setConfig(qorig)
    return res

CONTROLLER = 'simulation'
#when you are ready, replace the above line with this
#CONTROLLER = 'physical'
if CONTROLLER == 'simulation':
    from RobotControllerEmulator import kill_controller_threads,UR5WithGripperController
    controllerWorld = world.copy()
    kvis_ground_truth = ipyklampt.KlamptWidget(controllerWorld,width=600,height=400)
    robot_control_api = UR5WithGripperController(controllerWorld,randomize=True)
    VIS_UPDATE_FREQUENCY = 10
    VIS_UPDATE_ITERS = 0
    def _update_vis_hook():
        global VIS_UPDATE_ITERS
        VIS_UPDATE_ITERS += 1
        if VIS_UPDATE_ITERS % VIS_UPDATE_FREQUENCY == 0:
            #this is called inside the thread
            kvis_ground_truth.update()
    def start_controller():
        kill_controller_threads()
        robot_control_api.start(_update_vis_hook)
    kvis_ground_truth.set_camera({u'position': {u'y': 1.9298983866544108, u'x': 0.7498269220525384, u'z': 1.3093949571767591}, u'up': {u'y': 0.9488807720023056, u'x': 0.06236278376382235, u'z': -0.30941261080559734}, u'target': {u'y': 0.6599169321160283, u'x': -0.12092171623712519, u'z': 0.08376713045866419}})
else:
    from robot_api.RobotController import UR5WithGripperController
    robot_control_api = UR5WithGripperController(host='192.168.0.1')
    def start_controller():
        robot_control_api.start()
def update_vis_from_controller():
    q = robot_control_api.getConfig()
    robot.setConfig(controller_to_klampt(q))
    kvis.update()
def stop_controller():
    robot_control_api.stop()
def wait_controller(duration):
    t0 = time.time()
    tstart = t0
    while time.time()-tstart < duration:
        update_vis_from_controller()
        t1 = time.time()
        time.sleep(max(0,0.05-(t1-t0)))
        t0 = t1


In [3]:
if CONTROLLER == 'simulation':
    display(Markdown("## Ground truth world"))
    display(kvis_ground_truth)

controls = widgets.HBox([])
display(controls)

display(Markdown("## Planning world"))
display(kvis)

output_area = widgets.Output()
display(output_area)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

## Ground truth world

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdwb3NpdGlvbic6IHt1J3knOiAxLjkyOTg5ODM4NjY1NDQxMDgsIHUneCc6IDAuNzQ5ODI2OTIyMDUyNTM4NCwgdSd6JzogMS4zMDkzOTQ5NTcxNzY3NTnigKY=


HBox()

## Planning world

S2xhbXB0V2lkZ2V0KGNhbWVyYT17dSdwb3NpdGlvbic6IHt1J3knOiAxLjkyOTg5ODM4NjY1NDQxMDgsIHUneCc6IDAuNzQ5ODI2OTIyMDUyNTM4NCwgdSd6JzogMS4zMDkzOTQ5NTcxNzY3NTnigKY=


Output()

In [4]:
#Call this cell after you started the thread if you want to save CPU
stop_controller()

UR5WithGripperController emulation thread not started


In [5]:
#Call this cell to start the controller thread (the dot in the upper right of the notebook should flicker)
start_controller()
update_vis_from_controller()

Starting UR5WithGripperController emulation thread...
UR5GripperAPI thread #1 starting


In [6]:
#test control api
wait_controller(1)
q = robot_control_api.getConfig()
q[1] += 1.5
q[6] = 0
print "Setting first config"
robot_control_api.setConfig(q)
wait_controller(2)
q[1] -= 1.5
q[6] = 1
print "Setting second config"
robot_control_api.setConfig(q)
print "Waiting for 2 seconds"
wait_controller(2)
print "Done with movement"

Setting first config
Setting second config
Waiting for 2 seconds
Done with movement


In [7]:
#load camera and block marker poses from vision
#set up world to reflect sensor data


#IDs of objects in marker_poses.txt file
object_ids = [1,2,3]
id_to_object = dict((id,i) for (i,id) in enumerate(object_ids))
#known transforms from marker coordinates to object coordinates
marker_object_transforms = [(so3.identity(),(0,0,0.025))]*3

#load the calibrated camera transform
Tcamera = loader.load('RigidTransform','calibrated_camera_xform.txt')
print Tcamera

kvis.add_xform("CAMERA")
kvis.set_transform("CAMERA",*Tcamera)

#load the current markers from marker_poses.txt
def update_objects_from_sensors():
    object_poses = []
    with open('marker_poses.txt','r') as datafile:
        lines = datafile.readlines()
        for line in lines:
            if len(line.strip())==0:
                continue
            id,xform = line.split(None,1)
            T = loader.read('RigidTransform',xform)
            #this transpose is because the transform was written in column-major format, but loader assumed row-major
            T = (so3.inv(T[0]),T[1])
            object_poses.append((int(id),T))
    #print "Read object poses",object_poses
    #reset objects to be far away if they are not
    for i in xrange(world.numRigidObjects()):
        world.rigidObject(i).setTransform(*se3.identity())
    for (id,T) in object_poses:
        if id not in id_to_object:
            raise RuntimeError("Invalid object ID specified, must be one of "+str(object_ids))
        ind = id_to_object[id]
        Tobj_c = se3.mul(T,se3.inv(marker_object_transforms[ind]))
        Tobj = se3.mul(Tcamera,Tobj_c)
        world.rigidObject(ind).setTransform(*Tobj)

#emulate camera sensor readings by writing to marker_poses.txt
def emulate_camera_sensor():
    global world,marker_object_transforms,object_ids,Tcamera
    with open('marker_poses.txt','w') as datafile:
        for i in xrange(controllerWorld.numRigidObjects()):
            Tm_c = se3.mul(se3.inv(Tcamera),se3.mul(world.rigidObject(i).getTransform(),marker_object_transforms[i]))
            datafile.write(str(object_ids[i]) + '    ' + loader.write((so3.inv(Tm_c[0]),Tm_c[1]),'RigidTransform')+'\n')


([-0.0217268228463, -0.990605306817, 0.135015077952, -0.851489884476, -0.0524355829894, -0.521742739548, 0.523920720914, -0.126299785201, -0.842351139641], [0.105938110847, 0.114881052998, 1.07113035801])


In [8]:
from klampt.model import ik
from klampt.model import collide
from klampt.plan import robotplanning

#information about the end effector
ee_link = 7
#the driver responsible for changing the gripper value
ee_driver = 6
#the position of the center link
ee_localpos = (0.15,0,0)
#the direction pointing from the palm toward the fingers
ee_fwd_dir = (1,0,0)
#the direction pointing perpendicular to the fingers
ee_perpendicular_dir = (0,0,1)

#driver parameters
ee_driver_open_value = 0.0
ee_driver_closed_value = 0.723

def set_gripper_width(robot,q,width):
    """Helper: produces a configuration which is the same as q, except that the
    gripper is opened to the given width (in the range 0 (closed) to 1 (open))"""
    qold = robot.getConfig()
    robot.setConfig(q)
    robot.driver(ee_driver).setValue(ee_driver_open_value*width + (1.0-width)*ee_driver_closed_value)
    qres = robot.getConfig()
    robot.setConfig(qold)
    return qres

def feasible(robot,q):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - q: a configuration
    
    Out: True if q is feasible (collision free, in joint limits)
    """
    qmin,qmax = robot.getJointLimits()
    return True

def visible(robot,qstart,qend):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - qstart,qend: start and end configurations
    
    Out: True if the straight line joint space path between qstart / qend is feasible
    """
    return feasible(robot,qstart) and feasible(robot,qend)

def closest_ik_solution(robot,q,qcurrent):
    """TODO: complete me, if it helps
    
    In:
    - robot: a RobotModel instance
    - q: a candidate IK solution, which has joint angles in the range [-2pi,2pi]
    - qcurrent: the current configuration
    
    Out: a modified version of q that has the same joint angles modulo 2pi (hence, has the
    same layout in workspace), but is closer to qcurrent so that motion is minimized.
    """
    return q

def move_home(robot):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance

    Out: a list of milestones interpolating from the robot's current configuration to the home configuration.
      
      None can be returned if your planner cannot find a solution.
    """
    global qhome
    state = 'HOME'
    return [robot.getConfig(),qhome]


def find_pregrasp_config(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me for C.1
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a configuration q reaching the target pregrasp.
      None can be returned if there is no solution, but in all test cases there should be a solution.
    """
    
    rot = so3.mul(block_pose[0], so3.from_axis_angle(((0.0, 1.0, 0.0), math.pi/2)))
    trans = (block_pose[1][0], block_pose[1][1], block_pose[1][2] + ee_localpos[0]/2 + block_width + offset_height)
    obj = ik.objective(robot.link(ee_link), ref=None, R=rot, t=trans) 
    ik.solve(obj)
    
    return set_gripper_width(robot,robot.getConfig(),1)

def move_pregrasp(robot,block_pose,offset_height=0.06):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at qhome
    - block_pose: a transform of the center of the block
    - offset_height: the distance to hover above the block
    
    Out: a list of milestones interpolating from the robot's current configuration to the target pregrasp.
      
      None can be returned if your planner cannot find a solution.
    """
    qstart = robot.getConfig()
    qgrasp = find_pregrasp_config(robot,block_pose,offset_height)
    return [qstart,qgrasp]

def lower_and_close(robot,block_pose,close_width):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently posed at the pregrasp configuration
    - block_pose: a transform of the center of the block
    - close_width: the amount that the gripper should be closed
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving straight down (in Cartesian space), and then closing the robot's gripper.
    
    None can be returned if your planner cannot find a solution.
    """
    
    # Taken from Trishul Lab 3 ---
    qstart = robot.getConfig()
    
    rot = so3.mul(block_pose[0], so3.from_axis_angle(((0.0, 1.0, 0.0), math.pi/2)))
    trans = (block_pose[1][0], block_pose[1][1], block_pose[1][2] + ee_localpos[0]/2 + block_width)
    obj = ik.objective(robot.link(ee_link), ref=None, R=rot, t=trans)
    ik.solve(obj)
    
    qdown = robot.getConfig()
    qclose = set_gripper_width(robot,qdown,close_width)
    
    path = []
    path.append(qstart)
    path.append(qdown)
    path.append(qclose)
    print(path)
    return path
    # End Taken from Trishul Lab 3
    
    
def lift_and_place(robot,target_position):
    """
    TODO: complete me
    
    In:
    - robot: a RobotModel instance, with the robot currently gripping an object
    - target_position: the position of the target box
    
    Out: a list of milestones interpolating from the robot's current configuration
    moving 1) to lift the block up, 2) to the target position, and 3) opening the gripper
    
    None can be returned if your planner cannot find a solution.
    """
    
    # Copied from Trishul Lab 3
    qstart = robot.getConfig()
    
    start = robot.link(ee_link).getTransform()
    trans = (start[1][0], start[1][1], start[1][2] + 0.10)
    obj = ik.objective(robot.link(ee_link), ref=None, R=start[0], t=trans)
    ik.solve(obj)
    qup = robot.getConfig()
    
    obj = ik.objective(robot.link(ee_link), local=[ee_localpos], world=[target_position]) 
    ik.solve(obj)
    qbox = robot.getConfig()
    qopen = set_gripper_width(robot,qbox,1)
    return [qstart,qup,qbox,qopen]
    # End Copied from Trishul Lab 3

#     qstart = robot.getConfig()
#     qup = qstart[:]
#     qbox = qup[:]
#     qopen = set_gripper_width(robot,qbox,1)
#     return [qstart,qup,qbox,qopen]

In [9]:
#run this cell to bind your code to the visualization playback window. You should 
#not need to modify this code, unless you want to tweak the trajectory execution speed

#target parameters
target_position = (0.26,-0.15,1.2)

#block parameters
block_width = 0.05
block_names = [world.rigidObject(i).getName() for i in range(world.numRigidObjects())]
block_poses = [world.rigidObject(i).getTransform() for i in range(world.numRigidObjects())]

#gripper parameters
block_close_amount = 0.5

#the speed of trajectory execution: default 0.5 radian per second
#this may be tweaked, if desired
trajectory_execution_speed = 0.5

from klampt.model.trajectory import RobotTrajectory

#state can be 'HOME', 'PREGRASP', 'GRASPED', 'BOX'
state = 'HOME'

kvis.add_text("HUD1",1,1)
kvis.add_text("HUD2",1,5)
kvis.add_text("HUD3",1,9)
kvis.add_text("HUD4",1,13)
kvis.add_text("HUD5",1,17)
kvis.add_text("HUD6",1,21)
kvis.add_text("HUD7",1,25)

#the amount of time between refreshes
dt = 0.05
                
def execute_trajectory(traj):
    """Executes a trajectory on the controller.  You may or may not want to modify this."""
    global robot,controls
    for c in controls.children:
        c.disabled = True
        
    #send the trajectory
    dtraj = traj.discretize(dt)
    q0 = robot_control_api.getConfig()
    q0_traj = klampt_to_controller(traj.milestones[0])
    robot_control_api.setConfig(klampt_to_controller(traj.milestones[0]))
    t0 = traj.times[0]
    for t,q in zip(dtraj.times[1:],dtraj.milestones[1:]):
        wait_controller(t-t0)
        robot_control_api.setConfig(klampt_to_controller(q))
        t0 = t
    
    for c in controls.children:
        c.disabled = False

def trajectory_from_path(path):
    """TODO: perform a smoother path interpolation"""
    global robot
    times = [0.0]
    for i in range(1,len(path)):
        d = robot.distance(path[i-1],path[i])
        times.append(times[-1]+d/trajectory_execution_speed)
    print "Inferred path duration",times[-1]-times[0]
    return RobotTrajectory(robot,times,path)

def execute_path(path):
    execute_trajectory(trajectory_from_path(path))
    
def do_move_home(button):
    global state,robot,qhome
    output_area.clear_output()
    with output_area:
        try:
            
            milestones = move_home(robot)
            if milestones is not None:
                execute_path(milestones)
        except Exception:
            import traceback
            traceback.print_exc()
    state = 'HOME'

def do_sense(button):
    output_area.clear_output()
    with output_area:
        if CONTROLLER == 'simulation':
            emulate_camera_sensor()
        update_objects_from_sensors()
    kvis.update()
    
def do_move_pregrasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'HOME':
                print "WARNING: should do pregrasp from the home configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            selected_object = object_dropdown.value
            milestones = move_pregrasp(robot,world.rigidObject(selected_object).getTransform())
#             if qtgt is not None:      # Original Line
            if milestones is not None:  # Piazza Fix
                execute_path(milestones)
                state = 'PREGRASP'
        except Exception:
            import traceback
            traceback.print_exc()
        
            
def do_grasp(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'PREGRASP':
                print "WARNING: should do grasp from the pregrasp configuration"
            selected_object = object_dropdown.value
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lower_and_close(robot,world.rigidObject(selected_object).getTransform(),block_close_amount)
            if milestones is not None:
                execute_path(milestones)
                state = 'GRASPED'
        except Exception:
            import traceback
            traceback.print_exc()

def do_place(button):
    global state,robot
    output_area.clear_output()
    with output_area:
        try:
            if state != 'GRASPED':
                print "WARNING: should do place from the grasped configuration"
            robot.setConfig(controller_to_klampt(robot_control_api.getConfig()))
            milestones = lift_and_place(robot,target_position)
            if milestones is not None:
                execute_path(milestones)
                state = 'BOX'
        except Exception:
            import traceback
            traceback.print_exc()
        
object_dropdown = widgets.Dropdown(
    options=block_names,
    value=block_names[0],
    description='Object:'
)

move_home_button = widgets.Button(description="Move Home")
sense_button = widgets.Button(description="Sense")
move_pregrasp_button = widgets.Button(description="Move to Pregrasp")
grasp_button = widgets.Button(description="Grasp")
place_button = widgets.Button(description="Place")
move_home_button.on_click(do_move_home)
sense_button.on_click(do_sense)
move_pregrasp_button.on_click(do_move_pregrasp)
grasp_button.on_click(do_grasp)
place_button.on_click(do_place)

controls.children = (move_home_button,sense_button,object_dropdown,move_pregrasp_button,grasp_button,place_button)

## Written answers

Problem B

Problem C